In [1]:
%reset
%load_ext autoreload
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import scipy as sp

from scipy import signal
from scipy import io as sio

import neurodsp as ndsp
from neurodsp import spectral
from neurodsp import filt

from jupyterthemes import jtplot
jtplot.style()

import matplotlib.pyplot as plt
import seaborn as sns

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# Generating a Weak PING
Here, I modify several parameters at a time from Malerba et al. (2016) network model, following what I understood to be Paola's suggestion: 
<br>
> increase gmaxEE + increase direct current to inhibitory neurons + increase gmaxII to synchronize your inhibitory population without having to totally crank up the direct current + slightly increase the direct current to pyramidal neurons to achieve 6 Hz oscillation in the firing rate figures



## First attempt
| model params | new value | orig value | 
| --- | --- | --- | 
| p.gmaxII | 5/(60/100) = 8.3 | 3/(80/100) = 3.75 | 
| p.gmaxEE | (5/(60/100))/2 = 4.17| (5/(60/100))/10 = 0.83|
| p.Idc | 360 | 180 |
